# Choose the recruitment data file and split the data into applicable format


## Step 1 : find the recruitment data

The recruitment data generated by FEM model will be stored under one of the /input_# directory after each usage of the pipeline. Please find the directory that contains the recruitment data you prefer to use.


## Step 2: split the file
The recruitment data will be a txt file with title like "rec_task3_2022-07-04_1803.txt". It contains all the neuron recruitments and we need to extract the recruitment of Pudendal afferent, Pelvic afferent and SPN neurons from it.
For example, if the required data is inputs/input_1/rec_task3_2022-07-04_1803.txt, run the following code

In [13]:
# change directory and filename on a needs basis
directory = "input_1"
filename = "/rec_task3_2022-08-25_2042.txt"


recruitment = []
amplitude_data = []

for line in open(directory+filename,"r"):
    data = line.split()
    recruitment.append(data)
for afferent in recruitment:
    if afferent[0] == "Amps":
        amplitude_data = afferent[1].split(',')
    elif afferent[0] ==  "PelvicAff":
        pel_data = afferent[1].split(',')
    elif afferent[0] == "PelvicEff":
        spn_data = afferent[1].split(',')
    elif afferent[0] == "PudendalAff":
        pud_data = afferent[1].split(',')

# write the recruitment into separate files
def write_files(filename, data):
    with open(filename + '.txt', 'w') as fp:
        for item in data:
            # write each item on a new line
            fp.write("%s\n" % item)
        print(filename + ' Done')


# pudendal afferents

write_files("PudAff",pud_data)
write_files("PelAff",pel_data)
write_files("SPN",spn_data)
       
            

PudAff Done
PelAff Done
SPN Done


After running the above cell, you should be able to see three files under /inputs: 

+ PelAff.txt
+ PudAff.txt
+ SPN.txt


## Step3: change current mapping range in EES.py
go to ../neuralnetwork/code, open EES.py. Find two variables (should be around line 90~91): 

`self._minCur`, `self._maxCur`

in function `_load_rec_data()`


run the following cell to get the minimum current and maximum current for the mapping

In [14]:
print("MinCur: ", amplitude_data[0], '\n')
print("MaxCur: ", amplitude_data[-1])

MinCur:  0 

MaxCur:  1000


Change self._minCur, self._maxCur to the above values, save and exit EES.py file. 


In [7]:
#If desired in th4e configuration file, plot roots recruitment.
from os.path import join
import json

input_1 = "/home/jovyan/work/inputs/input_1"
input_2 = "/home/jovyan/work/inputs/input_2"

jfile = join(input_2, "config.json")
from os.path import join
with open(jfile, "r") as f:
    data = json.load(f)["pipeline_options"]

nodename = "function"
options = data[nodename]
print("configuration options for this node:")
for k, v in options.items():
    print("    %s: %s" % (k, v))
    
project_name = data["project_name"]
stim_amps=options["stim_amp_uA"]
stim_freq = options["stim_freq_Hz"]
start_volume=options["start_volume_ml"]
end_volume=options["end_volume_ml"]
sim_time=options["simulation_time_ms"]
repetitions=options["repetitions"]



configuration options for this node:
    stim_amp_uA: 500
    stim_freq_Hz: 50
    start_volume_ml: 20
    end_volume_ml: 20
    simulation_time_ms: 10000
    repetitions: 1


Now you are done with the recruitment data processing. 

Run the following cell to generate the command for running simulations using the configurations.

In [11]:
command = "python scripts/runBladderSim.py frwSimCat.txt " + str(sim_time) + ' ' + str(repetitions) + ' ' + str(stim_freq) + ' ' + str(stim_amps) +  " 0.0 0.0 0.0 1 "+ str(start_volume) + ' '+ str(end_volume)
print(command)

python scripts/runBladderSim.py frwSimCat.txt 10000 1 50 500 0.0 0.0 0.0 1 20 20


**Please copy the above command and go back to README for the next steps.**